In [1]:
import pickle
import pandas as pd
import os
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch
with torch.profiler.profile() as profiler:
        pass
from sklearn.preprocessing import MinMaxScaler

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
import wandb

wandb.login()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nickojelly (use `wandb login --relogin` to force relogin)


True

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")
    

Running on the GPU


In [4]:
os.chdir(r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\pytorch\New Model')

In [5]:
REBUILD_DATA = True


class GRV:
    # class to store training data

    # reading data from pickle
    file = open(r"DATA/total_list_new_box_index_fix.npy", "rb")
    data = pickle.load(file)
    # seperate out classes from inputs
    raceIDs, inputs, classes, prices, margins, betfairSP,names = zip(*data)
    raceIDs_float = [float(x) for x in raceIDs]
    # removing nan from inputs and convert to float
    inputs_df = pd.DataFrame(inputs)
    inputs_df = inputs_df.fillna(value=-1)
    inputs = inputs_df.values.tolist()
    # inputs = [[float(i) for i in j] for j in inputs]

    # data
    training_data = []




    def make_training_data(self):
        excluded = 0
        # for i in range(len(self.inputs)):
        self.raceIDs_t = torch.tensor(np.array(self.raceIDs_float))
        self.inputs_t = torch.tensor(np.array(self.inputs))
        self.classes_t = torch.tensor(np.array(self.classes))
        self.prices_t =torch.tensor(np.array(self.prices))
        self.margins_t = torch.tensor(np.array(self.margins))
        self.bsp_t = torch.tensor(np.array(self.betfairSP))
 

            # print(type(self.inputs[i]))
            #self.rIDs_t_test =
            # if 1270<i<1290:
            # #     print(i, self.raceIDs[i], self.names[i], self.classes[i])
            # if len(self.classes[i]) == 8:
            #     self.training_data.append(
            #         [
            #             torch.FloatTensor(self.inputs[i]),
            #             torch.FloatTensor(self.classes[i]),
            #             torch.FloatTensor(self.prices[i]),
            #             torch.FloatTensor(self.margins[i]),
            #             torch.FloatTensor(self.betfairSP[i]),
            #             torch.Tensor([float(self.raceIDs[i])])
            #         ]
            #     )
            #     if 1270<i<1290:
            #         print(i ,torch.Tensor([self.raceIDs_float[i]]).detach(), self.raceIDs_float[i], self.raceIDs[i])
            # else:
            #     print("failed")

        np.save("training_data.npy", self.training_data)
        print("excluded = ", excluded)


if REBUILD_DATA:
    grv = GRV()
    grv.make_training_data()


excluded =  0


In [6]:
test_class = grv.classes
test_class = list(test_class)
torch.tensor(np.array(test_class))

tensor([[8., 4., 6.,  ..., 7., 5., 1.],
        [6., 5., 1.,  ..., 3., 4., 2.],
        [6., 8., 5.,  ..., 7., 2., 1.],
        ...,
        [1., 8., 8.,  ..., 8., 5., 4.],
        [2., 3., 8.,  ..., 8., 5., 1.],
        [4., 3., 5.,  ..., 8., 2., 1.]], dtype=torch.float64)

In [7]:
test_class = grv.raceIDs_float
test_class = list(test_class)
x = torch.tensor(np.array(test_class))
x[1277].item()

140918776.0

In [8]:
print(grv.names[1285])
print(x[1285].item())


['MARODO', 'blank', 'MORTAL SIN', 'SUNDANCE LAD', 'RIVERSIDE MONS', 'MISS PIE', 'TALTARNI SHIRAZ', 'THUNDEROUS NIGHT']
140918784.0


In [9]:

# X = torch.stack([i[0] for i in training_data])
# Y = torch.stack([i[1] for i in training_data])
# P = torch.stack([i[2] for i in training_data])
# Y_m = softmin(torch.stack([i[3] for i in training_data])).float()
# bfSP = torch.stack([i[4] for i in training_data])
# rIDs = torch.tensor([float(x) for x in grv.raceIDs])
# new_rIDs = torch.stack([i[5] for i in training_data])

In [10]:
softmin = nn.Softmin(dim=1)

scaler = MinMaxScaler()


# dataset setup
training_data = grv.training_data

X = grv.inputs_t.clone().detach()
Y = grv.classes_t.clone().detach()
P = grv.prices_t.clone().detach()
Y_m = softmin(grv.margins_t).float().clone().detach()
bfSP = grv.bsp_t.clone().detach()
rIDs = grv.raceIDs_t.clone().detach()
# new_rIDs = torch.stack([i[5] for i in training_data])
len_dataset = len(X)
# Generate winner only class
Y_w = []
for i in Y:
    n = np.zeros(8)
    index = torch.argmin(i)
    n[index] = float(1)
    Y_w.append(n)

Y_w = torch.tensor([i for i in Y_w])

scaler.fit(X)
Xn = scaler.transform(X)
Xn = torch.tensor(Xn).float()

with open("normalizer.npy", "wb") as fp:   #Pickling
    
    pickle.dump(scaler, fp)

# Xn = nn.functional.normalize(X, dim = 1)
X = Xn.to(device)
Y_w = Y_w.to(device)


Y_m = Y_m.to(device)
P = P.to(device)
bfSP = bfSP.to('cuda')
bfSP = torch.nan_to_num(bfSP, nan=0)
my_dataset = TensorDataset(X, Y_m, P, bfSP,rIDs)
my_dataloader = DataLoader(my_dataset)
print(len(X))

C:\Users\Nick\AppData\Local\Temp\ipykernel_32404\2544570433.py:25: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:204.)
  Y_w = torch.tensor([i for i in Y_w])


72546


In [11]:
torch.set_printoptions(sci_mode=False)
for i,(ins,margins,prices,bsfp,race_IDs) in enumerate(my_dataloader):
    if race_IDs.item()==140918784.0:
        print(ins,margins,prices,bsfp.long(),race_IDs)
        print(torch.argmax(margins))
        break
    #break

tensor([[0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2941, 1.0000, 0.9939,
         0.9965, 0.9906, 0.9885, 0.9610, 0.9565, 0.4861, 0.1375, 0.1854, 0.0391,
         0.0476, 0.0500, 0.1049, 0.9925, 0.0214, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 1.0000, 0.9943, 0.9968, 0.9899, 0.9906, 0.9615,
         0.9582, 0.5397, 0.1383, 0.0862, 0.0385, 0.0213, 0.0323, 0.7523, 0.9939,
         0.1229, 1.0000, 0.9934, 0.0000, 0.9875, 0.9899, 0.0000, 0.9564, 0.4179,
         0.0251, 0.0000, 0.0109, 0.0488, 0.0250, 0.6622, 0.9922, 0.0214, 1.0000,
         0.9961, 0.0000, 0.9901, 0.9869, 0.0000, 0.9557, 0.3388, 0.1726, 0.0000,
         0.0110, 0.0213, 0.0215, 0.0830, 0.9948, 0.1630, 1.0000, 0.9942, 0.9960,
         0.9901, 0.9907, 0.9615, 0.9571, 0.4679, 0.0827, 0.1387, 0.0281, 0.0426,
         0.0538, 0.0931, 0.9

In [12]:
# have to pass in dataset to get_data (created above)
def make_loader(dataset, config, train=True):
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(config["validation_split"] * dataset_size))
    random_seed = 42
    np.random.seed(random_seed)
    np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]

    if train:
        dataset_sampler = SubsetRandomSampler(train_indices)
        loader = torch.utils.data.DataLoader(
            dataset=dataset,
            batch_size=config["batch_size"],
            shuffle=False,
            pin_memory=False,
            num_workers=0,
            sampler=dataset_sampler
        )
    else:
        dataset_sampler = SubsetRandomSampler(val_indices)
        loader = torch.utils.data.DataLoader(
            dataset=dataset,
            shuffle=False,
            pin_memory=False,
            num_workers=0,
            sampler=dataset_sampler,
        )

    return loader


In [13]:
def build_network(f1_layer_size, f2_layer_size, dropout, num_layers=2):

    if num_layers == 2:
        network = nn.Sequential(  # fully-connected, dual hidden layer
            nn.Linear(144, f1_layer_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(f1_layer_size, f2_layer_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(f2_layer_size, 8),
            nn.Softmax(dim=1),
        )

    else:
        network = nn.Sequential(  # fully-connected, dual hidden layer
            nn.Linear(144, f1_layer_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(f1_layer_size, f2_layer_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(f2_layer_size, f2_layer_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(f2_layer_size, 8),
            nn.Softmax(dim=1),
        )


    return network


In [14]:
def custom_MSE(output, target):
    sorts = torch.argsort(target)
    out = sorts.narrow(1,0,3)
    ohe = torch.nn.functional.one_hot(out, num_classes=8).sum(1)
    out_first3 = ohe*output
    target_ohe = ohe*target
    loss = torch.sum((((target_ohe+1)*abs(out_first3-target_ohe)+1)**2))
    # loss = torch.sum(((out_first3-target_ohe))**2)
    return loss
    


In [15]:
x = torch.tensor([[1,2,3,5,6]])
y =torch.tensor([[6,2,3,4,5]]) 


In [16]:
def custom_order_MSE(output, target):
    sorts = torch.argsort(target)
    print(sorts)
    out = sorts.narrow(1,0,3)
    ohe = torch.nn.functional.one_hot(out, num_classes=8).sum(1)
    out_first3 = ohe*output
    target_ohe = ohe*target
    loss = torch.mean(((abs(out_first3-target_ohe)*10)**2))
    # loss = torch.sum(((out_first3-target_ohe))**2)
    return loss
    

In [17]:
def loss_customiser(x,y,loss_func=nn.HuberLoss(reduction='none',delta=0.1)):
    sorts_t = torch.argsort(y)
    out = sorts_t.narrow(1,0,3)
    ohe = torch.nn.functional.one_hot(out, num_classes=8).sum(1)
    loss = loss_func(x,y)
    return torch.mean(loss)

In [18]:
def custom_l1(x=None,y=None,beta=0.1):
    loss_func=nn.SmoothL1Loss(reduction='none',beta=beta)
    sorts_t = torch.argsort(y)
    sorts_o = torch.argsort(x)
    diff_place = abs(sorts_t-sorts_o)
    out = sorts_t.narrow(1,0,3)
    ohe = torch.nn.functional.one_hot(out, num_classes=8).sum(1)
    loss = loss_func(x,y)*ohe
    return torch.mean(loss)

In [19]:
def custom_l1_place(x=None,y=None,beta=0.1):
    loss_func=nn.SmoothL1Loss(reduction='none',beta=beta)
    sorts_t = torch.argsort(y)
    sorts_o = torch.argsort(x)
    diff_place = abs(sorts_t-sorts_o)
    out = sorts_t.narrow(1,0,3)
    ohe = torch.nn.functional.one_hot(out, num_classes=8).sum(1)
    loss = loss_func(x,y)*diff_place
    return torch.mean(loss)

In [20]:
def custom_l1_place_1(x=None,y=None,beta=0.1):
    loss_func=nn.SmoothL1Loss(reduction='none',beta=beta)
    sorts_t = torch.argsort(y)
    weight_adj = 1/(sorts_t+0.5) # WOOOH MAGIC NUMBER BAD :(
    out = sorts_t.narrow(1,0,3)
    ohe = torch.nn.functional.one_hot(out, num_classes=8).sum(1)
    loss = loss_func(x,y)*weight_adj
    return torch.mean(loss)

In [21]:
def make(config, dataset):
    # Make the data

    train_loader = make_loader(dataset, config, train=True)
    test_loader = make_loader(dataset, config, train=False)
    # Make the model
    # model = Net().to(device)
    model = build_network(
        config["f1_layer_size"], config["f2_layer_size"], config["dropout"], config["num_layers"]
    )

    loss_functions = {
        "Huber":nn.HuberLoss(),
        "Huber_custom":loss_customiser,
        "l1_custom":custom_l1_place,
        "l1_custom_1stplace":custom_l1_place_1,
        "MSE":nn.MSELoss(),
        "L1":nn.SmoothL1Loss(reduction='mean', beta=config["l1_beta"]),
        "BCE":nn.CrossEntropyLoss(),
        "Custom":custom_MSE,
        "KL":nn.KLDivLoss(reduction='batchmean'),
        "NLL":nn.NLLLoss()
    }
    # Make the loss and optimizer
    #  criterion = nn.NLLLoss()
    loss_f = loss_functions[config['loss']]
    criterion = loss_f
    optimizer = config["optimizer"]

    if optimizer == "sgd":
        optimizer = optim.SGD(
            model.parameters(), lr=config["learning_rate"], momentum=0.9
        )
    if optimizer == "adam":
        optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])
    print(optimizer)
    if optimizer == "adamW":
        print("HERE")
    optimizer = optim.AdamW(model.parameters(), lr=config["learning_rate"])

    return model, train_loader, test_loader, criterion, optimizer


In [35]:
def test(model, test_loader, epoch):
    model.eval()
    classL, predL, maxL, correctL, priceP, priceR, bfPriceR, pred_odds, model_outputs, race_ID_list, delta_odds_list, prices_L = [], [], [], [], [], [], [], [], [], [], [], []
    # Run the model on some test examples
    with torch.no_grad():
        correct, total, max_sum, max_w_sum, profit, bfprofit, bfnotavail, bf_available,bf_only_bets, bf_only_profit, test_val,bf_sp_prices, zero_price = 0, 0, 0, 0, 0,0,0,0,0,0 ,0,0,0
        alt_bet_1, alt_bet_2 = 0,0
        value_pick_correct, value_pick_profit = 0, 0
        num_bets = 0
        for images, labels, prices, bfspPrices, raceID in test_loader:

            outputs = model(images)
            race_ID_list.append(raceID.item())

            #converts prices from tensor to list
            prices = prices[0,].tolist()
            bfspPrices = bfspPrices[0,].tolist()

            #gets the prediction and its confidence and the real class
            max, predicted = torch.max(outputs.data, 1)
            _, real = torch.max(labels.data, 1)

            #converts prediction from tensor to item
            prediction = predicted.item()
            real_item = real.item()


            #appends prediction and likelyhood to lists
            predL.append(prediction)
            maxL.append(max.item())

        
            total += labels.size(0)
            correct += prediction == real_item

            correctL.append(int(prediction == real_item))
            classL.append(real_item)

            priceR.append(prices[real_item])
            priceP.append(prices[prediction])
            prices_L.append(prices)
            bfPriceR.append(bfspPrices[real_item])
            # print(outputs.data.flatten().tolist())

            bf_odds_filled = []
            for i,odd in enumerate(bfspPrices):
                if odd:
                    bf_odds_filled.append(odd)
                else:
                    bf_odds_filled.append(prices[i])

            predicted_odds = [
                1 / ((x + 10**-7)) for x in outputs.data.flatten().tolist()
            ]

            delta_odds = [x - y for x, y in zip(bf_odds_filled, predicted_odds)]

            delta_odds_list.append(delta_odds)
            pred_odds.append(predicted_odds)
            model_outputs.append(outputs.data.flatten().tolist())

            if prices[real_item] > (predicted_odds[real_item]):
                value_pick_correct += 1
                value_pick_profit += prices[real_item]
                

            bets = [x > (y) for x, y in zip(prices, predicted_odds)]
            num_bets += sum(bets)

            value_pick_profit += -sum(bets)

            delta_pred = prices[prediction]-predicted_odds[prediction]
            alt_bet_1 -= 1+10*delta_pred
            alt_bet_2 -= 5


            if bfspPrices[prediction]:
                bf_only_bets += 1
                bf_only_profit -= 1
                bf_sp_prices += bfspPrices[prediction]
                test_val += 1

            if prediction == real_item:
                if prices[real_item]==0:
                    zero_price +=1
                    #print(raceID.item())
                max_sum += max
                profit += prices[real_item]
                if bfspPrices[real_item]:
                    bfprofit += bfspPrices[real_item]
                    bf_only_profit += bfspPrices[real_item]
                    alt_bet_1 += (1+10*delta_pred)*bfspPrices[real_item]
                    alt_bet_2 += 5*bfspPrices[real_item]
                    bf_available += 1
                    test_val -= 1
                else:
                    bfprofit += prices[real_item]*1.12
                    bfnotavail += 1
                    alt_bet_1 += (1+10*delta_pred)*prices[real_item]*1.12
                    alt_bet_2 += 5*prices[real_item]*1.12
            else:
                max_w_sum += max

            profit += -1
            bfprofit += -1

            # print(f"{correct=}")

        # print(f"Accuracy of the model on the {total} " +
        #       f"test images: {100 * correct / total}%" +
        #       f"profit: {profit}"+
        #       f"profit: {value_pick_profit}")

        wandb.log(
            {
                "test_accuracy": correct / total,
                "correct_conf": max_sum / correct,
                "incorrect_conf": (max_w_sum) / (total - correct),
                "profit": profit,
                "bfprofit": bfprofit,
                "bfnotavail": bfnotavail,
                "value_pick_roi": value_pick_profit / num_bets,
                "value_pick_correct": value_pick_correct,
                "num_bets_per": num_bets / total,
                "bf_avail": bf_available,
                "bfOnlyProfit": bf_only_profit,
                "bfOnlyBets": bf_only_bets,
                "test_val": test_val,
                "alt_bet_1":alt_bet_1,
                "alt_bet_2":alt_bet_2,
                "zero_price":zero_price
            }
        )

        if epoch%100==0:

            logdf = pd.DataFrame(
                data={
                    "class": classL,
                    "pred": predL,
                    "max": maxL,
                    "correct": correctL,
                    "priceR": priceR,
                    "priceP": priceP,
                    "bets": sum(bets),
                    "pred_odds": pred_odds,
                    "model_outputs": model_outputs,
                    "bfodds" : bfPriceR,
                    "raceID":race_ID_list,
                    "delta_odds":delta_odds_list,
                    "pricesL":prices_L
                    
                }
            )

            table = wandb.Table(dataframe=logdf)
            wandb.log({"table_key": table})
        # classCounts = logdf["class"].value_counts()
        # predCounts = logdf["pred"].value_counts()
        # boxplot = logdf.boxplot(column=['priceR'],by='correct')
        # print(classCounts, predCounts)
        # boxplot
        # plt.savefig("boxplot.png")
        # wandb.log({"boxplot":boxplot})




In [23]:
def train_log(loss, example_ct, epoch):
    # Where the magic happens
    wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)
    #print(f"Loss after " + str(example_ct).zfill(5) + f" examples: {loss:.3f}")

In [24]:
def model_saver(model, optimizer, epoch, loss):
    
    pathtofolder = "C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model"
    model_name = wandb.run.name
    isExist = os.path.exists(
        f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/"
    )
    if isExist:
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "loss": loss,
            },
            f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_{epoch}.pt",
        )
    else:
        print("created path")
        os.makedirs(
            f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/"
        )
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "loss": loss,
            },
            f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_{epoch}.pt",
        )


In [25]:
def train(model, loader,test_loader, criterion, optimizer, config):
    # Tell wandb to watch what the model gets up to: gradients, weights, and more!
    wandb.watch(model, criterion, log="all", log_freq=10)

    # Run training and track with wandb
    total_batches = len(loader) * config.epochs
    example_ct = 0  # number of examples seen
    batch_ct = 0

    raw_inputs = True
    if config['loss'] == "KL":

        for epoch in tqdm(range(config.epochs)):
            for _, (images, labels, _ , _, _) in enumerate(loader):

                loss = train_batch_lsftmax(images, labels, model, optimizer, criterion, btch_count=batch_ct, raw_inputs=True)
                example_ct +=  len(images)
                batch_ct += 1

                # Report metrics every 25th batch
                if ((batch_ct + 1) % 250) == 0:
                    train_log(loss, example_ct, epoch)
                    

            if epoch %10 ==0:
                test(model,test_loader, epoch)
                model_saver(model,optimizer,epoch,loss)

    else:
        for epoch in tqdm(range(config.epochs)):
            for _, (images, labels, _ , _, _) in enumerate(loader):

                loss = train_batch(images, labels, model, optimizer, criterion, btch_count=batch_ct, raw_inputs=True)
                example_ct +=  len(images)
                batch_ct += 1

                # Report metrics every 25th batch
                if ((batch_ct + 1) % 250) == 0:
                    train_log(loss, example_ct, epoch)

            if epoch %10 ==0:
                test(model,test_loader, epoch)
                model_saver(model, optimizer, epoch, loss)

def train_batch(images, labels, model, optimizer, criterion, btch_count=0, raw_inputs=True, beta=0.1):
    images, labels = images, labels
    

    # Forward pass ➡
    outputs = model(images)
    loss = criterion(outputs, labels.float())
    
    # Backward pass ⬅
    optimizer.zero_grad()
    loss.backward()

    # Step with optimizer
    optimizer.step()

    return loss


def train_batch_lsftmax(images, labels, model, optimizer, criterion, btch_count=0, raw_inputs=True):
    images, labels = images, labels
    

    # Forward pass ➡
    outputs = model(images)
    loss = criterion(F.log_softmax(outputs), labels.float())
    
    # Backward pass ⬅
    optimizer.zero_grad()
    loss.backward()

    # Step with optimizer
    optimizer.step()

    return loss

In [26]:
def model_pipeline(config=None,prev_model=None):
    dataset = my_dataset
    # tell wandb to get started
    with wandb.init(project="new customs 71.7K", config=config):
      # access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss", summary="min")
      wandb.define_metric("test_accuracy", summary="max")
      wandb.define_metric("bfprofit", summary="max")
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)

      # make the model, data, and optimization problem
      model, train_loader, test_loader, criterion, optimizer = make(config, dataset)

      prev_model = r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\pytorch\New Model\good models\worthy-sweep-6_290.pt'
      if prev_model:
        checkpoint = torch.load(prev_model, map_location="cuda:0")
        print("here")
        model.load_state_dict(checkpoint['model_state_dict'])
        # optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
      model = model.to(device)
      #optimizer = optimizer.to(device)
      print(model)

      # and use them to train the model
      train(model, train_loader,test_loader, criterion, optimizer, config)

      # and test its final performance
      #test(model, test_loader)

    return model

In [27]:
sweep_config = {"method": "random"}

metric = {"name": "profit", "goal": "maximize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"value": "adamW"},
    "f1_layer_size": {"values": [256]},
    "f2_layer_size": {"values": [64]},
    "dropout": {"values": [0.3, 0.4, 0.5]},
    "len_data": {"value": len_dataset},
}

sweep_config["parameters"] = parameters_dict

parameters_dict.update(
    {
        "epochs": {"values": [1000]},
        "validation_split": {"value": 0.1},
        "loss": {
            "values": [ "L1"],
            # "values": ["Huber", "MSE", "L1", "BCE", "Custom", "KL"]
            # 'value': 'l1_custom'
        },
        "num_layers": {"values": [2]},
    }
)

parameters_dict.update(
    {
        "learning_rate": {
            # a flat distribution between 0 and 0.1
            "distribution": "uniform",
            "min": 0.00011,
            "max": 0.001,
        },
        "l1_beta": {
            # a flat distribution between 0 and 0.1
            "distribution": "uniform",
            "min": 0,
            "max": 1,
        },
        "batch_size": {
            'values': [1000]
            # "values": [32, 64, 128, 360, 720]
            # 'values':[4,8,16,32,64,128,360]
        },
    }
)

import pprint

pprint.pprint(sweep_config)


sweep_config


{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'profit'},
 'parameters': {'batch_size': {'values': [1000]},
                'dropout': {'values': [0.3, 0.4, 0.5]},
                'epochs': {'values': [1000]},
                'f1_layer_size': {'values': [256]},
                'f2_layer_size': {'values': [64]},
                'l1_beta': {'distribution': 'uniform', 'max': 1, 'min': 0},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.001,
                                  'min': 0.00011},
                'len_data': {'value': 72546},
                'loss': {'values': ['L1']},
                'num_layers': {'values': [2]},
                'optimizer': {'value': 'adamW'},
                'validation_split': {'value': 0.1}}}


{'method': 'random',
 'metric': {'name': 'profit', 'goal': 'maximize'},
 'parameters': {'optimizer': {'value': 'adamW'},
  'f1_layer_size': {'values': [256]},
  'f2_layer_size': {'values': [64]},
  'dropout': {'values': [0.3, 0.4, 0.5]},
  'len_data': {'value': 72546},
  'epochs': {'values': [1000]},
  'validation_split': {'value': 0.1},
  'loss': {'values': ['L1']},
  'num_layers': {'values': [2]},
  'learning_rate': {'distribution': 'uniform', 'min': 0.00011, 'max': 0.001},
  'l1_beta': {'distribution': 'uniform', 'min': 0, 'max': 1},
  'batch_size': {'values': [1000]}}}

In [36]:

wandb_config_static = {'batch_size': 360, 'dropout': 0.3, 'epochs': 1000, 'f1_layer_size': 256, 'f2_layer_size': 64 , 'learning_rate': 0.0001, 'loss': 'L1', 'l1_beta':0.1, "len_data": len_dataset, 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
model = model_pipeline(config=wandb_config_static)
# wandb.init()
# model = model_pipeline(config=wandb_config_static, prev_model=r'C:\Users\Nick\Documents\GitHub\grvmodel\Python\pytorch\New Model\good models\vocal-sweep-80_90.pt')

  0%|          | 0/1000 [00:00<?, ?it/s]

{'batch_size': 360, 'dropout': 0.3, 'epochs': 1000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'learning_rate': 0.0001, 'loss': 'L1', 'l1_beta': 0.1, 'len_data': 72546, 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
1000
{'batch_size': 360, 'dropout': 0.3, 'epochs': 1000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'learning_rate': 0.0001, 'loss': 'L1', 'l1_beta': 0.1, 'len_data': 72546, 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
adamW
HERE
here
Sequential(
  (0): Linear(in_features=144, out_features=256, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.3, inplace=False)
  (3): Linear(in_features=256, out_features=64, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.3, inplace=False)
  (6): Linear(in_features=64, out_features=8, bias=True)
  (7): Softmax(dim=1)
)


  0%|          | 1/1000 [00:20<5:38:04, 20.30s/it]

created path


100%|██████████| 1000/1000 [53:30<00:00,  3.21s/it] 


alt_bet_1,▄▆█▆▅▆▇█▇▇▆▅█▆▇▅█▅▆▄▆▅▄▄▇▆▆▆▃▂▂▆▄▃▂▁▁▅▁▃
alt_bet_2,▁▄▆█▃▆▇█▅▆▆▄█▆▆▆▆▅▆▅▄▄▄▂▅▆▅▄▃▂▂▅▇▂▅▂▁▇▁▄
bfOnlyBets,▅▃▃▄▅▃▄▅▄▃▂▅▅▃▁▃▄▃▃▂▄▄▁▄▅▄▃▆▅▅▂▆▇▇▆▄▂▆██
bfOnlyProfit,▁▅▆▇▅▇▆▇▅▆▆▅▇▆▆▆▆▆▇▆▅▅▆▃▆▆▆▅▄▄▄▆▇▄▇▄▃█▄▆
bf_avail,▁▇▇█▆▇▇▇▅▅▅▆▆▆▄▅▅▅▅▅▄▄▄▂▅▄▄▂▃▃▃▄▄▂▄▃▁▃▂▂
bfnotavail,▄▆▆█▅▄▇▆▆▆▆▅▆▄█▇▆▆▆▆▄▅▄▄▅▅▅▄▄▅▂▃▅▁▂▃▆▃▃▂
bfprofit,▁▄▆█▃▆▇█▅▆▆▄█▆▆▆▆▅▆▅▄▄▄▂▅▆▅▄▃▂▂▅▇▂▅▂▁▇▁▄
correct_conf,▁▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇█▇
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
incorrect_conf,▁▅▅▅▅▅▆▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▆▇▇▇▇▇▇▇▇▇▇▇█▇▇█▇
loss,▇█▆▆▆▃▆▅▆▆█▅█▆▆▅▃▃▃▄▃▄▄▇▃▅▅▆▅▂▆▃▂▂▅▂▅▆▂▁


In [37]:
#model_pipeline(config)

sweep_id = wandb.sweep(sweep_config, project="grv_priced_sweep_custom_new_box")
CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=100)


Create sweep with ID: s56m9tsl
Sweep URL: https://wandb.ai/nickojelly/grv_priced_sweep_custom_new_box/sweeps/s56m9tsl


wandb: Agent Starting Run: ch83m2jb with config:
wandb: 	batch_size: 1000
wandb: 	dropout: 0.3
wandb: 	epochs: 1000
wandb: 	f1_layer_size: 256
wandb: 	f2_layer_size: 64
wandb: 	l1_beta: 0.06931838167221116
wandb: 	learning_rate: 0.000829931728485932
wandb: 	len_data: 72546
wandb: 	loss: L1
wandb: 	num_layers: 2
wandb: 	optimizer: adamW
wandb: 	validation_split: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


  0%|          | 0/1000 [00:00<?, ?it/s]

{'batch_size': 1000, 'dropout': 0.3, 'epochs': 1000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'l1_beta': 0.06931838167221116, 'learning_rate': 0.000829931728485932, 'len_data': 72546, 'loss': 'L1', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
1000
{'batch_size': 1000, 'dropout': 0.3, 'epochs': 1000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'l1_beta': 0.06931838167221116, 'learning_rate': 0.000829931728485932, 'len_data': 72546, 'loss': 'L1', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
adamW
HERE
here
Sequential(
  (0): Linear(in_features=144, out_features=256, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.3, inplace=False)
  (3): Linear(in_features=256, out_features=64, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.3, inplace=False)
  (6): Linear(in_features=64, out_features=8, bias=True)
  (7): Softmax(dim=1)
)


  0%|          | 1/1000 [00:20<5:38:58, 20.36s/it]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

{'batch_size': 1000, 'dropout': 0.3, 'epochs': 1000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'l1_beta': 0.2754543312270695, 'learning_rate': 0.0007876668038121364, 'len_data': 72546, 'loss': 'L1', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
1000
{'batch_size': 1000, 'dropout': 0.3, 'epochs': 1000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'l1_beta': 0.2754543312270695, 'learning_rate': 0.0007876668038121364, 'len_data': 72546, 'loss': 'L1', 'num_layers': 2, 'optimizer': 'adamW', 'validation_split': 0.1}
adamW
HERE
here
Sequential(
  (0): Linear(in_features=144, out_features=256, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.3, inplace=False)
  (3): Linear(in_features=256, out_features=64, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.3, inplace=False)
  (6): Linear(in_features=64, out_features=8, bias=True)
  (7): Softmax(dim=1)
)


  0%|          | 1/1000 [00:18<5:08:44, 18.54s/it]

created path


 56%|█████▌    | 560/1000 [39:08<18:59,  2.59s/it]  